In [1]:
import sys
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os


In [2]:
conf = SparkConf().setMaster("local[*]").setAppName("mnm-rdd-counts")

# SparkContext 생성
sc = SparkContext(conf=conf)

24/12/03 16:49:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
directory = os.path.join(os.getcwd(), "data")  # 현재 디렉토리의 data 폴더
filename  = "mnm_dataset.csv"
file_path = os.path.join(directory, filename)
lines = sc.textFile("file:///" + file_path.replace("\\", "/"))

In [4]:
lines.take(5)

['State,Color,Count', 'TX,Red,20', 'NV,Blue,66', 'CO,Blue,79', 'OR,Blue,71']

In [5]:
header = lines.first()

In [6]:
datas = lines.filter(lambda row: row != header)

In [7]:
mnms_datas = datas.map(lambda row: row.split(","))

In [8]:
mnms_datas.collect()[:3]

[['TX', 'Red', '20'], ['NV', 'Blue', '66'], ['CO', 'Blue', '79']]


blue mnms 추출

In [9]:
blue_data = mnms_datas.filter(lambda row: row[1]=="Blue")
blue_data.collect()

[['NV', 'Blue', '66'],
 ['CO', 'Blue', '79'],
 ['OR', 'Blue', '71'],
 ['WY', 'Blue', '16'],
 ['AZ', 'Blue', '75'],
 ['CO', 'Blue', '52'],
 ['CO', 'Blue', '95'],
 ['CO', 'Blue', '98'],
 ['CA', 'Blue', '13'],
 ['NV', 'Blue', '50'],
 ['TX', 'Blue', '49'],
 ['CA', 'Blue', '34'],
 ['UT', 'Blue', '97'],
 ['AZ', 'Blue', '59'],
 ['OR', 'Blue', '29'],
 ['OR', 'Blue', '54'],
 ['CO', 'Blue', '52'],
 ['WA', 'Blue', '59'],
 ['WY', 'Blue', '59'],
 ['WY', 'Blue', '53'],
 ['WA', 'Blue', '42'],
 ['WA', 'Blue', '37'],
 ['NV', 'Blue', '49'],
 ['NM', 'Blue', '15'],
 ['CA', 'Blue', '39'],
 ['UT', 'Blue', '48'],
 ['AZ', 'Blue', '82'],
 ['CO', 'Blue', '14'],
 ['WA', 'Blue', '18'],
 ['CO', 'Blue', '89'],
 ['TX', 'Blue', '80'],
 ['WA', 'Blue', '84'],
 ['NM', 'Blue', '46'],
 ['AZ', 'Blue', '16'],
 ['TX', 'Blue', '68'],
 ['CA', 'Blue', '99'],
 ['WY', 'Blue', '80'],
 ['CO', 'Blue', '44'],
 ['CO', 'Blue', '90'],
 ['TX', 'Blue', '19'],
 ['CA', 'Blue', '65'],
 ['TX', 'Blue', '98'],
 ['NM', 'Blue', '98'],
 ['AZ', 'Bl

총 합계 구하기

In [10]:
count_sum = mnms_datas.map(lambda row: int(row[2])).sum()
print(f"Total Count: {count_sum}")

Total Count: 5500035


도시별 합계 구하기

In [11]:
city_rdd = mnms_datas.map(lambda row: (row[0], int(row[2])))
city_total_data = city_rdd.reduceByKey(lambda a,b: a+b)
print(city_total_data.collect())

[('TX', 556493), ('CO', 556302), ('WY', 538975), ('CA', 561184), ('NM', 553454), ('NV', 548477), ('OR', 542169), ('WA', 557227), ('AZ', 546812), ('UT', 538942)]


In [10]:
sc.stop()